## Backpropagation exercise
Now you're going to implement the backprop algorithm for a network trained on the graduate school admission data. You should have everything you need from the previous exercises to complete this one.

Your goals here:

* Implement the forward pass.
* Implement the backpropagation algorithm.
* Update the weights.

In [8]:
# Prepare data
import numpy as np
import pandas as pd

admissions = pd.read_csv('datasets/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
features_test.T

,48,50,80,84,98,110,120,122,133,148,...,309,312,315,317,328,356,368,375,386,396
gre,-1.278605,0.452749,0.972155,-0.759199,0.972155,0.799020,-0.586063,-0.586063,-0.759199,-0.932334,...,-1.278605,0.625884,-2.490553,1.664697,-0.066657,0.972155,-0.066657,-0.239793,1.318426,-0.239793
gpa,-2.390908,1.235263,-1.287291,0.552071,-1.339844,-0.814312,0.919944,-1.418674,-0.814312,-1.261014,...,-1.077078,0.998773,-1.444950,0.630901,-0.131120,-0.315056,1.603135,0.263029,1.235263,-0.919418
rank_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
rank_2,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
rank_3,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
rank_4,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [9]:
import numpy as np

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)


In [18]:
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))
        print("output: {}".format(output))
        
        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y - output
        print("error: {}".format(error))

        # TODO: Calculate error term for the output unit
        output_error_term = error * output * (1 - output)
        print("output_error_term: {}".format(output_error_term))

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        print("hidden_error: {}".format(hidden_error))

        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        print("hidden_error_term: {}".format(hidden_error_term))
        
        # TODO: Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        print("del_w_hidden_output: {}".format(del_w_hidden_output))
        print("x: {}".format(x))
        del_w_input_hidden += hidden_error_term * x[:, None]
        print("del_w_input_hidden: {}".format(del_w_input_hidden))
        print("---")
        break

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records
    break

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

output: 0.5707935369711756
error: 0.42920646302882437
output_error_term: 0.1051505510491229
hidden_error: [ 0.06406563 -0.03500296]
hidden_error_term: [ 0.01513006 -0.00696304]
del_w_hidden_output: [ 0.06494334  0.02881197]
x: [ 0.97215519  0.44696493  1.          0.          0.          0.        ]
del_w_input_hidden: [[ 0.01470876 -0.00676915]
 [ 0.0067626  -0.00311223]
 [ 0.01513006 -0.00696304]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
---
Prediction accuracy: 0.500
